# Notebook A: Mechalis Menton Fitting

### Setup imports

In [ ]:
import pandas as pd
from scipy.optimize import curve_fit 
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
phenol_hrp_to_peroxide_ratio_df = pd.read_csv('../data/phenol_hrp_to_peroxide_ratio.csv')

phenol_hrp_to_peroxide_ratio_df

### Define a function for finding MM fit (without uncertainty)

In [ ]:
# Define the Michaelis-Menten equation
def michaelis_menten(t, Vmax, Km):
    return 1 - Vmax * t / (Km + t)

def find_mm_fit(df, ratio): 
    # define the columns to use
    column_1, column_2 = f'{ratio} hrp:peroxide phe 1', f'{ratio} hrp:peroxide phe 2'

    time = df['time'].values
    concentration = df[[column_1, column_2]].mean(axis=1).values

    # Use curve_fit to find the best-fit parameters
    initial_guess = [1, 1]  # You can adjust these initial guesses if needed
    bounds = ([0, 0], [np.inf, np.inf])  # Vmax >= 0, Km >= 0

    params, covariance = curve_fit(michaelis_menten, time, concentration, p0=initial_guess, bounds=bounds)

    # fitted parameters
    Vmax, Km = params

    # find the fit data points
    fitted_data = michaelis_menten(time, *params)

    return Vmax, Km, fitted_data

### Generate MM fits

In [ ]:
Vmax_0_5_to_1, Km_0_5_to_1, data_0_5_to_1 = find_mm_fit(phenol_hrp_to_peroxide_ratio_df, '0.5:1')
phenol_hrp_to_peroxide_ratio_df['0.5:1 MM fit'] = data_0_5_to_1

Vmax_1_to_1, Km_1_to_1, data_1_to_1 = find_mm_fit(phenol_hrp_to_peroxide_ratio_df, '1:1')
phenol_hrp_to_peroxide_ratio_df['1:1 MM fit'] = data_1_to_1

Vmax_2_to_1, Km_2_to_1, data_2_to_1 = find_mm_fit(phenol_hrp_to_peroxide_ratio_df, '2:1')
phenol_hrp_to_peroxide_ratio_df['2:1 MM fit'] = data_2_to_1

phenol_hrp_to_peroxide_ratio_df

### Plot the data and MM fits

In [ ]:
time = phenol_hrp_to_peroxide_ratio_df['time'].values

# Plotting the original data and the fitted curve
plt.figure(figsize=(8, 6))

# display 0.5:1 data and fit
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['0.5:1 hrp:peroxide phe 1'], color='black', label='0.5:1 trial 1')
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['0.5:1 hrp:peroxide phe 2'], color='black', marker='s', label='0.5:1 trial 2')
plt.plot(time, phenol_hrp_to_peroxide_ratio_df['0.5:1 MM fit'], color='black', linestyle='--', label=f'0.5:1 Vmax={Vmax_0_5_to_1:.3f}, Km={Km_0_5_to_1:.3f}')

# display 1:1 data and fit
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['1:1 hrp:peroxide phe 1'], color='green', label='1:1 trial 1')
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['1:1 hrp:peroxide phe 2'], color='green', marker='s', label='1:1 trial 2')
plt.plot(time, phenol_hrp_to_peroxide_ratio_df['1:1 MM fit'], color='green', linestyle='--', label=f'1:1 Vmax={Vmax_1_to_1:.3f}, Km={Km_1_to_1:.3f}')

# display 2:1 data and fit
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['2:1 hrp:peroxide phe 1'], color='blue', label='2:1 trial 1')
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['2:1 hrp:peroxide phe 2'], color='blue', marker='s', label='2:1 trial 2')
plt.plot(time, phenol_hrp_to_peroxide_ratio_df['2:1 MM fit'], color='blue', linestyle='--',  label=f'2:1 Vmax={Vmax_2_to_1:.3f}, Km={Km_2_to_1:.3f}')

plt.xlabel('Time (minutes)')
plt.ylabel('Relative Phenol Concentration')
plt.title('Effect of HRP:Hydrogen Peroxide Ratio on Phenol Removal')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

### Calculate the uncertainty of Vmax and Km

In [ ]:
def find_mm_fit_uncertainty(df, ratio): 
    # define the columns to use
    column_1, column_2 = f'{ratio} hrp:peroxide phe 1', f'{ratio} hrp:peroxide phe 2'

    time = df['time'].values
    concentration_1 = df[[column_1]].values.flatten()
    concentration_2 = df[[column_2]].values.flatten()

    # Use curve_fit to find the best-fit parameters
    initial_guess = [1, 1]  # You can adjust these initial guesses if needed
    bounds = ([0, 0], [np.inf, np.inf])  # Vmax >= 0, Km >= 0

    params_1, covariance = curve_fit(michaelis_menten, time, concentration_1, p0=initial_guess, bounds=bounds)
    params_2, covariance = curve_fit(michaelis_menten, time, concentration_2, p0=initial_guess, bounds=bounds)

    # fitted parameters
    Vmax_1, Km_1 = params_1
    Vmax_2, Km_2 = params_2

    average_Vmax = (Vmax_1 + Vmax_2) / 2
    average_Km = (Km_1 + Km_2) / 2

    std_Vmax = np.std([Vmax_1, Vmax_2])
    std_Km = np.std([Km_1, Km_2])

    average_params = [average_Vmax, average_Km]

    # find the fit data points
    fitted_data = michaelis_menten(time, *average_params)

    return average_Vmax, average_Km, std_Vmax, std_Km, fitted_data

find_mm_fit_uncertainty(phenol_hrp_to_peroxide_ratio_df, '2:1')

### Generate MM fits

In [ ]:
Vmax_0_5_to_1, Km_0_5_to_1, Vmax_std_0_5_to_1, Km_std_0_5_to_1, data_0_5_to_1 = find_mm_fit_uncertainty(phenol_hrp_to_peroxide_ratio_df, '0.5:1')
phenol_hrp_to_peroxide_ratio_df['0.5:1 MM fit'] = data_0_5_to_1

Vmax_1_to_1, Km_1_to_1, Vmax_std_1_to_1, Km_std_1_to_1, data_1_to_1 = find_mm_fit_uncertainty(phenol_hrp_to_peroxide_ratio_df, '1:1')
phenol_hrp_to_peroxide_ratio_df['1:1 MM fit'] = data_1_to_1

Vmax_2_to_1, Km_2_to_1, Vmax_std_2_to_1, Km_std_2_to_1, data_2_to_1 = find_mm_fit_uncertainty(phenol_hrp_to_peroxide_ratio_df, '2:1')
phenol_hrp_to_peroxide_ratio_df['2:1 MM fit'] = data_2_to_1

phenol_hrp_to_peroxide_ratio_df

### Plot the data and MM fits

In [ ]:
time = phenol_hrp_to_peroxide_ratio_df['time'].values

# Plotting the original data and the fitted curve
plt.figure(figsize=(8, 6))

# display 0.5:1 data and fit
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['0.5:1 hrp:peroxide phe 1'], color='black', label='0.5:1 trial 1')
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['0.5:1 hrp:peroxide phe 2'], color='black', marker='s', label='0.5:1 trial 2')
plt.plot(time, phenol_hrp_to_peroxide_ratio_df['0.5:1 MM fit'], color='black', linestyle='--', label=f'0.5:1 Vmax={Vmax_0_5_to_1:.3f}, Km={Km_0_5_to_1:.3f}')

# display 1:1 data and fit
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['1:1 hrp:peroxide phe 1'], color='green', label='1:1 trial 1')
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['1:1 hrp:peroxide phe 2'], color='green', marker='s', label='1:1 trial 2')
plt.plot(time, phenol_hrp_to_peroxide_ratio_df['1:1 MM fit'], color='green', linestyle='--', label=f'1:1 Vmax={Vmax_1_to_1:.3f}, Km={Km_1_to_1:.3f}')

# display 2:1 data and fit
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['2:1 hrp:peroxide phe 1'], color='blue', label='2:1 trial 1')
plt.scatter(time, phenol_hrp_to_peroxide_ratio_df['2:1 hrp:peroxide phe 2'], color='blue', marker='s', label='2:1 trial 2')
plt.plot(time, phenol_hrp_to_peroxide_ratio_df['2:1 MM fit'], color='blue', linestyle='--',  label=f'2:1 Vmax={Vmax_2_to_1:.3f}, Km={Km_2_to_1:.3f}')

plt.xlabel('Time (minutes)')
plt.ylabel('Relative Phenol Concentration')
plt.title('Effect of HRP:Hydrogen Peroxide Ratio on Phenol Removal')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig('../figures/peroxide_ratio_on_phenol.png', dpi=300, bbox_inches='tight')

plt.show()

### Save the parameters and their uncertainties

In [ ]:
ratio_col = ['0.5 to 1', '1 to 1', '2 to 1']
Vmax_col = [
    f'{Vmax_0_5_to_1:.4f} ± {Vmax_std_0_5_to_1:.4f}', 
    f'{Vmax_1_to_1:.4f} ± {Vmax_std_1_to_1:.4f}', 
    f'{Vmax_2_to_1:.4f} ± {Vmax_std_2_to_1:.4f}'
]

Km_col = [
    f'{Km_0_5_to_1:.4f} ± {Km_std_0_5_to_1:.4f}', 
    f'{Km_1_to_1:.4f} ± {Km_std_1_to_1:.4f}', 
    f'{Km_2_to_1:.4f} ± {Km_std_2_to_1:.4f}'
]

# Create a new DataFrame to display the results
summary_df = pd.DataFrame({'Ratio': ratio_col, 'Vmax': Vmax_col, 'Km': Km_col})

# save the summary to a csv file
summary_df.to_csv('../results/hrp_to_peroxide_ratio_params.csv', index=False)

summary_df
